In [ ]:
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode()
from utils import read_json
import numpy as np
import pandas as pd

### Data  Manipulation

In [ ]:
transactions = read_json('transactions.json')
projects_json = read_json('projects.json')
wallet2user = read_json('wallet2user.json')

In [ ]:
projects_wallets_to_ids = dict()
projects_wallets = set()
for x in projects_json:
    projects_wallets_to_ids["0x%s" % x['wallet']] = x['id'] 
    projects_wallets |= {"0x%s" % x['wallet']}

In [ ]:
investors = []
projects = []
timestamps = []
EXP_invested = []
for investor in transactions['from_to_amount']:
    for potential_project in transactions['from_to_amount'][investor]:
        if potential_project in projects_wallets:
            # Is a project
            for investment in transactions['from_to_amount'][investor][potential_project]:
                investors.append(investor)
                projects.append(potential_project)
                timestamps.append(investment['timestamp'])
                EXP_invested.append(investment['balance'])            

In [ ]:
# Investors, projects, TIMESTAMPS, EXP
sorted_investments = sorted(zip(investors, projects, timestamps, EXP_invested), key= lambda x: x[2])


In [ ]:
# Graphs and table
graph1 = []
table1 = []
index1 = []
N = len(list(set(investors)))
c= ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(0, 360, N)]
for i, investor in enumerate(list(set(investors))):
    if investor in wallet2user:
        investor_name = wallet2user[investor]['user']
    else:
        investor_name = investor
    invests = [t for t in sorted_investments if t[0]==investor]
    invests_projects = list({t[1] for t in invests})

    for project in invests_projects:
        if project in projects_wallets_to_ids:
            project_name = projects_wallets_to_ids[project]
        else:
            project_name = project
        
        x_axis = [t[2] for t in sorted_investments if t[0]==investor and t[1]==project and t[3]>10]
        y_axis = [t[3] for t in sorted_investments if t[0]==investor and t[1]==project and t[3]>10]
        if x_axis: 
            graph1.append(
                plotly.graph_objs.Scatter(
                    x=x_axis,
                    y=y_axis, 
                    name='%s, Project%s' % (investor_name, project_name),
                    marker=dict(color=c[i])
                )
            )
    
    # Append investor name
    index1.append(investor_name)
    
    # Get total balance and invested
    EXP_balance = wallet2user[investor]['balance']
    EXP_invested = sum([t[3] for t in sorted_investments if t[0]==investor]) 
    
    table1.append([EXP_balance, EXP_invested])

# Sort table by balance
sorted_table = sorted(zip(table1, index1), key=lambda x: x[0][0])[::-1]
table1 = []
index1 = []
for x in sorted_table:
    table1.append(x[0])
    index1.append(x[1])
    
graph2 = []
N = len(list(projects_wallets))
c = ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(0, 360, N)]
for i, project in enumerate(list(projects_wallets)):
    if project in projects_wallets_to_ids:
        project_name = projects_wallets_to_ids[project]
    else:
        project_name = project

    x_axis = [t[2] for t in sorted_investments if t[1]==project]
    y_axis = [t[3] for t in sorted_investments if t[1]==project]
    y_axis = [sum(y_axis[:p+1]) for p, y in enumerate(y_axis)]
    if x_axis and y_axis[-1] > 10: 
            graph2.append(
                plotly.graph_objs.Scatter(
                    x=x_axis,
                    y=y_axis, 
                    name='Project%s' % (project_name),
                    marker=dict(color=c[i])
                )
            )

# Top EXP left table
balances_users = [(wallet2user[x]['balance'], wallet2user[x]['user']) for x in wallet2user
                  if all(y in wallet2user[x] for y in ['user', 'balance'])]
# Pick top 20
sorted_balances_users = sorted(balances_users, key=lambda x:x[0])[::-1][:20]
table2 = [x[0] for x in sorted_balances_users]
index2 = [x[1] for x in sorted_balances_users]
# Add total
table2.append(sum(table2))
index2.append('TOTAL')

# EXP Invested by each Investor over time  (> 10EXP) 

In [ ]:
layout = go.Layout(
    title='EXP invested by each investor over time (per project) (> 10EXP)', 
    xaxis={'title' : 'Time'}, 
    yaxis={'title' : 'EXP'}
)
fig = go.Figure(
    data=graph1, 
    layout=layout
)
plotly.offline.iplot(fig)

# Total EXP invested in each Project over Time (> 10EXP)

In [ ]:
layout = go.Layout(
    title='Total EXP invested in each Project over time (> 10EXP)', 
    xaxis={'title' : 'Time'}, 
    yaxis={'title' : 'EXP'}
)
fig = go.Figure(
    data=graph2, 
    layout=layout
)
plotly.offline.iplot(fig)

# Top Investors (EXP Balance and Invested)

In [ ]:
print("Top Investors (EXP Balance and Invested)")
pd.DataFrame(
    data=table1,
    index = index1,
    columns=['EXP Balance', 'EXP Invested']
)

# Top 20 Balances 

In [ ]:
print("Top 20 Balances")
pd.DataFrame(
    data=table2,
    index = index2,
    columns=['EXP Balance']
)